### import numpy as np
import pandas as pd
# import tensorflow as tf

import cudf
import cuml
# from cuml.preprocessing import StandardScaler, MinMaxScaler

from cuml.decomposition import PCA
# from cuml.metrics import mean_squared_error


import numpy as np
import pandas as pd
import biom
import zipfile
import os
from biom import Table
from gemelli.rpca import rpca
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler


In [2]:
meta_path = "./redbiom_adrc_wolr2_fecal_v2.tsv"
meta = pd.read_csv(meta_path, sep='\t')

print(meta.shape)
print(meta.head(5))


(13436, 7)
                  #SampleID  qiita_study_id  host_age host_age_units  \
0  12675.FIT076E3CAL.170122           12675     40.00          years   
1  12142.820081881.2.159050           12142     77.05          years   
2  12142.820036321.4.158997           12142     60.80          years   
3  12142.820023241.6.158977           12142     70.49          years   
4       11666.G0281R.164064           11666     61.00          years   

  host_body_site diagnosis apoe  
0   UBERON:feces       NaN  NaN  
1   UBERON:feces       NaN  NaN  
2   UBERON:feces       NaN  NaN  
3   UBERON:feces       NaN  NaN  
4   UBERON:feces       NaN  NaN  


In [5]:
zip_file_path = './redbiom_adrc_wolr2_fecal_v2.biom.zip'

extract_to_path = './' 
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

extracted_files = os.listdir(extract_to_path)
print(f"Extracted files: {extracted_files}")

biom_file = None
for file in extracted_files:
    if file.endswith('.biom'):
        biom_file = os.path.join(extract_to_path, file)
        break

if biom_file:
    print(f"Loading .biom file: {biom_file}")
    
    table = biom.load_table(biom_file)
    
    biom_df = pd.DataFrame(table.matrix_data.toarray(), 
                      index=table.ids(axis='observation'), 
                      columns=table.ids(axis='sample'))
    
    print(biom_df.head())
else:
    print("No .biom file found in the extracted files.")


Extracted files: ['redbiom_adrc_wolr2_fecal_v2.tsv', '.ipynb_checkpoints', 'redbiom.ipynb', 'redbiom_adrc_wolr2_fecal_v2.biom', 'redbiom_adrc_wolr2_fecal_v2.biom.zip', '__MACOSX', 'redbiom_pca_cuml.ipynb']
Loading .biom file: ./redbiom_adrc_wolr2_fecal_v2.biom
            10283.JS.1.14.2015.160409  10283.LS.2.23.2015.159711  \
G000005825                        1.0                        0.0   
G000006175                        0.0                        0.0   
G000006605                        0.0                        0.0   
G000006725                        0.0                        0.0   
G000006745                        0.0                        2.0   

            11129.NH001.JJK.St.160942  11129.NH002.JJK.St.160942  \
G000005825                        0.0                        0.0   
G000006175                        0.0                        0.0   
G000006605                        8.0                        1.0   
G000006725                        0.0                     

In [6]:
def classify_feature(row):
    unique_values = row.unique()
    if len(unique_values) == 2 and set(unique_values).issubset({0, 1}):
        return 'binary'
    else:
        return 'numerical'

biom_df['feature_type'] = biom_df.apply(classify_feature, axis=1)

binary_features = biom_df[biom_df['feature_type'] == 'binary']
numerical_features = biom_df[biom_df['feature_type'] == 'numerical']

binary_features = binary_features.drop(columns=['feature_type'])
numerical_features = numerical_features.drop(columns=['feature_type'])

print(f"Number of binary features: {binary_features.shape[0]}")
print(f"Number of numerical features: {numerical_features.shape[0]}")


Number of binary features: 1568
Number of numerical features: 13795


In [ ]:
scaled_d = numerical_features.copy()
scaler = MinMaxScaler()

scaled_d[numerical_features.columns] = scaler.fit_transform(numerical_features)
scaled = scaler.fit_transform(numerical_features.T).T

# print("Scaled Numerical Dataset:")
# print(scaled.head())
# print(scaled.shape) # (13795, 13436)

In [ ]:
reduced_dim = 512
pca_model = PCA(n_components=reduced_dim, svd_solver='full')
df_pca = pca_model1.fit_transform(df_1)

df_reconstructed = pca_model1.inverse_transform(df1_pca)


In [4]:
# df_1 size: 637 * 20628
# Range: [0.0, 327369100.00000006]
# df_2 size: 744 * 17354
# Range: [0.0, 774804400.0]

In [5]:
# Unscaled dataset

In [6]:
reduced_dim = 128
pca_model1 = PCA(n_components=reduced_dim, svd_solver='full')
pca_model2 = PCA(n_components=reduced_dim, svd_solver='full')
df1_pca = pca_model1.fit_transform(df_1)
df2_pca = pca_model2.fit_transform(df_2)

df1_reconstructed = pca_model1.inverse_transform(df1_pca)
df2_reconstructed = pca_model2.inverse_transform(df2_pca)

In [7]:
print(f'df1_pca size: {df1_pca.shape[0]} * {df1_pca.shape[1]}')
print(f'df2_pca size: {df2_pca.shape[0]} * {df2_pca.shape[1]}')
print(f'df1_reconstructed size: {df1_reconstructed.shape[0]} * {df1_reconstructed.shape[1]}')
print(f'df2_reconstructed size: {df2_reconstructed.shape[0]} * {df2_reconstructed.shape[1]}')

df1_pca size: 637 * 128
df2_pca size: 744 * 128
df1_reconstructed size: 637 * 20628
df2_reconstructed size: 744 * 17354


In [8]:
mse_1 = mean_squared_error(df_1, df1_reconstructed)
mse_2 = mean_squared_error(df_2, df2_reconstructed)
print(f'----For {metab_1_path}:----')
print(f'Original size: {df_1.shape[0]} * {df_1.shape[1]}')
print(f'Reduced size: {df1_pca.shape[0]} * {df1_pca.shape[1]}')
print(f'Reconstructed mse: {mse_1}')

print(f'----For {metab_2_path}:----')
print(f'Original size: {df_2.shape[0]} * {df_2.shape[1]}')
print(f'Reduced size: {df2_pca.shape[0]} * {df2_pca.shape[1]}')
print(f'Reconstructed mse: {mse_2}')


----For metadata/metab_df_1_processed.csv:----
Original size: 637 * 20628
Reduced size: 637 * 128
Reconstructed mse: 224157915.67511678
----For metadata/metab_df_2_processed.csv:----
Original size: 744 * 17354
Reduced size: 744 * 128
Reconstructed mse: 5773051384.7409315


In [9]:
# MSE of Unscaled datasets with PCA
# ----For unscaled metadata/metab_df_1_processed.csv:----
# Original size: 637 * 20628
# Reduced size: 637 * 512
# Reconstructed mse: 2161562.807968613
# ----For unscaled metadata/metab_df_2_processed.csv:----
# Original size: 744 * 17354
# Reduced size: 744 * 512
# Reconstructed mse: 198845405.87344676


# ----For unscaled metadata/metab_df_1_processed.csv:----
# Original size: 637 * 20628
# Reduced size: 637 * 128
# Reconstructed mse: 224157915.67511678
# ----For unscaled metadata/metab_df_2_processed.csv:----
# Original size: 744 * 17354
# Reduced size: 744 * 128
# Reconstructed mse: 5773051384.7409315

In [10]:
# Scaled Dataset

In [11]:
scaler1 = StandardScaler()
scaler2 = StandardScaler()

scaled_1 = scaler1.fit_transform(df_1)
scaled_2 = scaler2.fit_transform(df_2)


In [12]:
print(f'scaled_1 size: {scaled_1.shape[0]} * {scaled_1.shape[1]}')
print(f'Range: [{scaled_1.min().min()}, {scaled_1.max().max()}]')
print(f'scaled_2 size: {scaled_2.shape[0]} * {scaled_2.shape[1]}')
print(f'Range: [{scaled_2.min().min()}, {scaled_2.max().max()}]')


scaled_1 size: 637 * 20628
Range: [-6.3458035046688135, 25.219040425837033]
scaled_2 size: 744 * 17354
Range: [-4.502650986590355, 27.258026340878086]


In [13]:
# scaled_1 size: 637 * 20628
# Range: [-6.3458035046688135, 25.219040425837033]
# scaled_2 size: 744 * 17354
# Range: [-4.502650986590355, 27.258026340878086]

In [14]:
reduced_dim_s = 1024
pca_scaled1 = PCA(n_components=reduced_dim_s, svd_solver='full')
pca_scaled2 = PCA(n_components=reduced_dim_s, svd_solver='full')
scaled1_pca = pca_scaled1.fit_transform(scaled_1)
scaled2_pca = pca_scaled2.fit_transform(scaled_2)

scaled1_reconstructed = pca_scaled1.inverse_transform(scaled1_pca)
scaled2_reconstructed = pca_scaled2.inverse_transform(scaled2_pca)

In [15]:
print(f'scaled1_pca size: {scaled1_pca.shape[0]} * {scaled1_pca.shape[1]}')
print(f'scaled2_pca size: {scaled2_pca.shape[0]} * {scaled2_pca.shape[1]}')
print(f'scaled1_reconstructed size: {scaled1_reconstructed.shape[0]} * {scaled1_reconstructed.shape[1]}')
print(f'scaled2_reconstructed size: {scaled2_reconstructed.shape[0]} * {scaled2_reconstructed.shape[1]}')


scaled1_pca size: 637 * 1024
scaled2_pca size: 744 * 1024
scaled1_reconstructed size: 637 * 20628
scaled2_reconstructed size: 744 * 17354


In [16]:
scaled_mse_1 = mean_squared_error(scaled_1, scaled1_reconstructed)
scaled_mse_2 = mean_squared_error(scaled_2, scaled2_reconstructed)
print(f'----For scaled {metab_1_path}:----')
print(f'Original size: {scaled_1.shape[0]} * {scaled_1.shape[1]}')
print(f'Reduced size: {scaled1_pca.shape[0]} * {scaled1_pca.shape[1]}')
print(f'Reconstructed mse: {scaled_mse_1}')

print(f'----For scaled {metab_2_path}:----')
print(f'Original size: {scaled_2.shape[0]} * {scaled_2.shape[1]}')
print(f'Reduced size: {scaled2_pca.shape[0]} * {scaled2_pca.shape[1]}')
print(f'Reconstructed mse: {scaled_mse_2}')


----For scaled metadata/metab_df_1_processed.csv:----
Original size: 637 * 20628
Reduced size: 637 * 1024
Reconstructed mse: 5.159855415010912e-29
----For scaled metadata/metab_df_2_processed.csv:----
Original size: 744 * 17354
Reduced size: 744 * 1024
Reconstructed mse: 1.012961906688683e-28


In [17]:
# ----For scaled metadata/metab_df_1_processed.csv:----
# Original size: 637 * 20628
# Reduced size: 637 * 128
# Reconstructed mse: 0.31091092994883607
# ----For scaled metadata/metab_df_2_processed.csv:----
# Original size: 744 * 17354
# Reduced size: 744 * 128
# Reconstructed mse: 0.451898048789962

# ----For scaled metadata/metab_df_1_processed.csv:----
# Original size: 637 * 20628
# Reduced size: 637 * 256
# Reconstructed mse: 0.14692968198374712
# ----For scaled metadata/metab_df_2_processed.csv:----
# Original size: 744 * 17354
# Reduced size: 744 * 256
# Reconstructed mse: 0.2537748722853918

# ----For scaled metadata/metab_df_1_processed.csv:----
# Original size: 637 * 20628
# Reduced size: 637 * 512
# Reconstructed mse: 0.01765819702708299
# ----For scaled metadata/metab_df_2_processed.csv:----
# Original size: 744 * 17354
# Reduced size: 744 * 512
# Reconstructed mse: 0.05859996044394567

In [ ]:
# minmaxscaler

In [18]:
minmax1 = MinMaxScaler()
minmax2 = MinMaxScaler()

minmax_1 = minmax1.fit_transform(df_1)
minmax_2 = minmax1.fit_transform(df_2)


In [19]:
print(f'scaled_1 size: {scaled_1.shape[0]} * {scaled_1.shape[1]}')
print(f'Range: [{scaled_1.min().min()}, {scaled_1.max().max()}]')
print(f'scaled_2 size: {scaled_2.shape[0]} * {scaled_2.shape[1]}')
print(f'Range: [{scaled_2.min().min()}, {scaled_2.max().max()}]')


scaled_1 size: 637 * 20628
Range: [-6.3458035046688135, 25.219040425837033]
scaled_2 size: 744 * 17354
Range: [-4.502650986590355, 27.258026340878086]


In [20]:
reduced_dim_s = 512
pca_scaled1 = PCA(n_components=reduced_dim_s, svd_solver='full')
pca_scaled2 = PCA(n_components=reduced_dim_s, svd_solver='full')
scaled1_pca = pca_scaled1.fit_transform(scaled_1)
scaled2_pca = pca_scaled2.fit_transform(scaled_2)

scaled1_reconstructed = pca_scaled1.inverse_transform(scaled1_pca)
scaled2_reconstructed = pca_scaled2.inverse_transform(scaled2_pca)

In [21]:
print(f'scaled1_pca size: {scaled1_pca.shape[0]} * {scaled1_pca.shape[1]}')
print(f'scaled2_pca size: {scaled2_pca.shape[0]} * {scaled2_pca.shape[1]}')
print(f'scaled1_reconstructed size: {scaled1_reconstructed.shape[0]} * {scaled1_reconstructed.shape[1]}')
print(f'scaled2_reconstructed size: {scaled2_reconstructed.shape[0]} * {scaled2_reconstructed.shape[1]}')


scaled1_pca size: 637 * 512
scaled2_pca size: 744 * 512
scaled1_reconstructed size: 637 * 20628
scaled2_reconstructed size: 744 * 17354


In [22]:
scaled_mse_1 = mean_squared_error(scaled_1, scaled1_reconstructed)
scaled_mse_2 = mean_squared_error(scaled_2, scaled2_reconstructed)
print(f'----For scaled {metab_1_path}:----')
print(f'Original size: {scaled_1.shape[0]} * {scaled_1.shape[1]}')
print(f'Reduced size: {scaled1_pca.shape[0]} * {scaled1_pca.shape[1]}')
print(f'Reconstructed mse: {scaled_mse_1}')

print(f'----For scaled {metab_2_path}:----')
print(f'Original size: {scaled_2.shape[0]} * {scaled_2.shape[1]}')
print(f'Reduced size: {scaled2_pca.shape[0]} * {scaled2_pca.shape[1]}')
print(f'Reconstructed mse: {scaled_mse_2}')


----For scaled metadata/metab_df_1_processed.csv:----
Original size: 637 * 20628
Reduced size: 637 * 512
Reconstructed mse: 0.01765819702708299
----For scaled metadata/metab_df_2_processed.csv:----
Original size: 744 * 17354
Reduced size: 744 * 512
Reconstructed mse: 0.05859996044394567
